In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

file_path = r"../data/hotel_bookings.csv"
data = pd.read_csv(file_path)
#  查看数据的行、列
print(data.shape)

(119390, 32)


In [6]:
# 查看数据非空数据量
data.count()

# NAN统计,county、agent、company、children中这四列出现nan数据
data.isna().sum() 

# # 数据整体描述
# data.describe()

# # 数据信息
# data.info()

# # 简单查看一下数据
# data.head()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

### 数据缺失值查询与处理 
通过describe()方法看出：
children 这一列少了4个值，且均值，4分位值均为0，因此填充为0值
country、agent、company：缺失值较多，分别为488，16340，112593。因此考虑去掉这些列，或者单独拿出来进行分析

In [36]:
# # df1 为去除nan列数据
df1 = data.drop(["country", "agent", "company"],axis=1)
df1.shape

# 填充nan
df1["children"].fillna(0,inplace=True)

# 查看填充后的结果,无nan列
df1.isna().sum()
print(df1.shape)

'''均值填充含nan的列
for column in list(df.columns[df.isnull().sum() > 0]):

mean_val = df[column].mean()

df[column].fillna(mean_val, inplace=True)'''
""
# list_col = list(data.columns[data.isnull().sum()>0])
# for col in  list_col:
#     print(col,list_col.index(col),"type:",type(col))

(119390, 29)


''

### 数据查重并处理

In [46]:
# 查询重复行数,此方法不算第一次出现“重复行”，
# 只拿后面的行往前比，所以第一次出现的内容就不算重复行，比如一行重复了3次，那↓方法反馈sum()=2
print(df1.duplicated().sum())

# 查看哪些记录出现重复值
rep_rows = df1[df1.duplicated(keep=False)].shape[0]
print(rep_rows)
df1[df1.duplicated(keep=False)]

33792
42532


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,A,0,No Deposit,0,Transient,98.00,0,1,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,A,0,No Deposit,0,Transient,98.00,0,1,Check-Out,2015-07-03
12,Resort Hotel,0,68,2015,July,27,1,0,4,2,...,E,0,No Deposit,0,Transient,97.00,0,3,Check-Out,2015-07-05
15,Resort Hotel,0,68,2015,July,27,1,0,4,2,...,E,0,No Deposit,0,Transient,97.00,0,3,Check-Out,2015-07-05
21,Resort Hotel,0,72,2015,July,27,1,2,4,2,...,A,1,No Deposit,0,Transient,84.67,0,1,Check-Out,2015-07-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119352,City Hotel,0,63,2017,August,35,31,0,3,3,...,D,0,No Deposit,0,Transient-Party,195.33,0,2,Check-Out,2017-09-03
119353,City Hotel,0,63,2017,August,35,31,0,3,3,...,D,0,No Deposit,0,Transient-Party,195.33,0,2,Check-Out,2017-09-03
119354,City Hotel,0,63,2017,August,35,31,0,3,3,...,D,0,No Deposit,0,Transient-Party,195.33,0,2,Check-Out,2017-09-03
119372,City Hotel,0,175,2017,August,35,31,1,3,1,...,A,0,No Deposit,0,Transient,82.35,0,1,Check-Out,2017-09-04
